In [28]:
import pandas as pd
from Bio import SeqIO
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [29]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction


In [30]:
# Correct path to your directory containing FASTA files
# fasta_dir = r"C:\Users\FEDGEN-LAB1-SYS7\OneDrive\Desktop\Mercy Akinwale\consensus_sequences"
fasta_dir = r"/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences"
# List to store sequence data
sequences_data = []

# Loop through each FASTA file in the directory
for fasta_file in os.listdir(fasta_dir):
    if fasta_file.endswith(".fa") or fasta_file.endswith(".fasta"):
        file_path = os.path.join(fasta_dir, fasta_file)
        print(file_path)
        record_seq=""
        record_id=""
        for n,record in enumerate(SeqIO.parse(file_path, "fasta")):
            if (n==0):
                record_id=record.id
            record_seq=record_seq+str(record.seq)
        print(record_id)
        sequences_data.append({
            "sequence_id": record_id,
            "sequence": str(record_seq)
        })

# Create a DataFrame from the sequence data
df = pd.DataFrame(sequences_data)

# Display the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(df.head())

# inspect the DataFrame's structure
print("\nDataFrame Information:")
print(df.info())


/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence8.fa
ERR029411_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence24.fa
ERR012425_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence10.fa
ERR012227_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence14.fa
ERR012269_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence20.fa
ERR012330_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence30.fa
ERR012494_1.fastq
/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/consensus_sequences/consensus_sequence15.fa
ERR012274_1.fastq
/opt/anaconda3/envs/M

In [31]:
# Paths to my data
labels_file_path = r'/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/csv_dataset_files/quninine_phenotype_dataset.csv'

# Function to read FASTA files from a directory and extract sequences
def read_fasta_from_dir(dir_path):
    sequences = {}
    for fasta_file in os.listdir(dir_path):
        if fasta_file.endswith(".fa") or fasta_file.endswith(".fasta"):
            file_path = os.path.join(dir_path, fasta_file)
            for record in SeqIO.parse(file_path, "fasta"):
                sequences[record.id] = str(record.seq)
    return sequences

# Function to one-hot encode the sequence
def one_hot_encode(seq):
    print("---------------"+str(len(seq)))
    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
    # return np.array([mapping[nuc] for nuc in seq.upper() if nuc in mapping])
    return np.array([mapping[nuc] for n,nuc in enumerate(seq.upper()) if ((n<1000000))])

# Read sequences from the FASTA directory
sequences = read_fasta_from_dir(fasta_dir)


In [32]:
import os

# Debug: Check if the file exists
if os.path.exists(labels_file_path):
    print(f"File exists: {labels_file_path}")
else:
    print(f"File not found: {labels_file_path}")


File exists: /opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/csv_dataset_files/quninine_phenotype_dataset.csv


In [33]:
# Try reading with different encodings
for encoding in ['utf-8', 'ISO-8859-1', 'latin1', 'cp1252']:
    try:
        labels_df = pd.read_csv(labels_file_path, encoding=encoding)
        print(f"Successfully read with encoding: {encoding}")
        break
    except Exception as e:
        print(f"Failed with encoding {encoding}: {e}")


Successfully read with encoding: utf-8


In [34]:
# Debug: Check the sequences read
print(f"Total sequences read: {len(sequences)}")
for seq_id, seq in list(sequences.items())[:5]:  # Print first 5 sequences
    print(f"ID: {seq_id}, Sequence: {seq[:50]}...")

# Read labels from CSV file
labels_df = pd.read_csv(labels_file_path)
labels_dict = labels_df.set_index('sequence_id').to_dict()['label']  # Assuming 'label' column for binary classification

# Debug: Check the labels read
print(f"Total labels read: {len(labels_dict)}")
for seq_id, label in list(labels_dict.items())[:5]:  # Print first 5 labels
    print(f"ID: {seq_id}, Label: {label}")


Total sequences read: 45
ID: ERR029411_1.fastq, Sequence: TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccct...
ID: NC_037280.1, Sequence: aaccctaaaccctaaaccctaaaccctaaaccctaaaccctaaacctaaa...
ID: NC_000521.4, Sequence: TAAACCCTAAATCTCTAAACCCTAAAGCTATACCTAAACCCTGAAGGTTA...
ID: NC_004318.2, Sequence: aaccctaaaccctgaaccctaaaccctaaaccctgaaccctgaaccctaa...
ID: NC_004326.2, Sequence: ctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccc...
Total labels read: 31
ID: ERR012494_1.fastq, Label: Sensitive
ID: ERR012519_1.fastq, Label: Sensitive
ID: ERR012425_1.fastq, Label: Sensitive
ID: ERR012424_1.fastq, Label: Sensitive
ID: ERR029095_1.fastq, Label: Resistant


In [35]:
# Prepare the sequences and labels for model training
encoded_sequences = []
labels = []


df_dict = df.to_dict("records")

for seq_dict in df_dict:
    if seq_dict['sequence_id'] in labels_dict:
        # Encode the sequence and flatten it
        encoded_sequences.append(one_hot_encode(seq_dict['sequence'].upper()).flatten())
        # Append the corresponding label(s) from the dictionary
        labels.append(labels_dict[seq_dict['sequence_id']].split(','))

# Convert labels to a suitable format for training/prediction
# Assuming binary classification with a single label per sample:
labels = [label[0] if isinstance(label, list) else label for label in labels]
labels = np.array(labels)

# Convert encoded sequences to a numpy array
print([f.shape for f in encoded_sequences])
encoded_sequences = np.array(encoded_sequences)


---------------23292141
---------------23287178
---------------23292541
---------------23292457
---------------23289447
---------------23270800
---------------23287492
---------------23286572
---------------23274363
---------------23292461
---------------23285585
---------------23285407
---------------23292472
---------------23292287
---------------23286900
---------------23251041
---------------23255091
---------------23292399
---------------23292372
---------------23265735
---------------23292244
---------------23289777
---------------23252337
---------------23258039
---------------23289229
---------------23292393
---------------23286531
---------------23292541
---------------23255999
[(4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (4000000,), (40

In [36]:
print (encoded_sequences)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [37]:
print(labels)

['Resistant' 'Sensitive' 'Resistant' 'Sensitive' 'Sensitive' 'Sensitive'
 'Resistant' 'Sensitive' 'Sensitive' 'Resistant' 'Sensitive' 'Sensitive'
 'Resistant' 'Resistant' 'Sensitive' 'Resistant' 'Resistant' 'Sensitive'
 'Sensitive' 'Sensitive' 'Resistant' 'Sensitive' 'Resistant' 'Sensitive'
 'Resistant' 'Resistant' 'Sensitive' 'Sensitive' 'Sensitive']


In [38]:
# Debug: Check the encoded sequences
print(f"Encoded sequences shape: {encoded_sequences.shape}")
if encoded_sequences.size > 0:
    print(f"First encoded sequence: {encoded_sequences[0]}")


Encoded sequences shape: (29, 4000000)
First encoded sequence: [0 0 0 ... 0 1 0]


In [39]:
encoded_sequences.shape

(29, 4000000)

In [40]:
np.array(labels)

array(['Resistant', 'Sensitive', 'Resistant', 'Sensitive', 'Sensitive',
       'Sensitive', 'Resistant', 'Sensitive', 'Sensitive', 'Resistant',
       'Sensitive', 'Sensitive', 'Resistant', 'Resistant', 'Sensitive',
       'Resistant', 'Resistant', 'Sensitive', 'Sensitive', 'Sensitive',
       'Resistant', 'Sensitive', 'Resistant', 'Sensitive', 'Resistant',
       'Resistant', 'Sensitive', 'Sensitive', 'Sensitive'], dtype='<U9')

In [41]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import statistics
from sklearn.exceptions import NotFittedError

# Assuming encoded_sequences and labels have been correctly prepared
if len(encoded_sequences) > 0 and len(labels) > 0:
    X_train, X_test, y_train, y_test = train_test_split(
        encoded_sequences, 
        labels, 
        test_size=0.2, 
        random_state=42
    )

    # Ensure y_train and y_test are 1D arrays
    y_train = np.array(y_train).ravel()
    y_test = np.array(y_test).ravel()

    # Print to debug shape and unique labels
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    print("Unique labels in y_train:", np.unique(y_train))

    param_grid = {
        'n_estimators': [2, 4, 8],
        'max_depth': [1, 5, 10, 15],
        'min_samples_split': [2, 5, 10]
    }

    kf = KFold(n_splits=5)

    ls_accuracy = []
    ls_precision = []
    ls_recall = []
    ls_f1 = []
    ls_roc_auc = []

    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        print("#####################################################")
        print("----- CV --- ", (i + 1))

        # Create a new RandomForestClassifier instance for each fold
        clf = RandomForestClassifier(random_state=42)

        X_train_cv = X_train[train_index]
        X_test_cv = X_train[test_index]

        y_train_cv = y_train[train_index]
        y_test_cv = y_train[test_index]

        # Perform grid search, which fits the model
        grid_search = GridSearchCV(clf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
        grid_search.fit(X_train_cv, y_train_cv)  # Model is fitted during this process

        # Print best parameters and best score
        print("Best parameters found: ", grid_search.best_params_)
        print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

        # Use the best estimator from grid search (this is the fitted model)
        best_clf = grid_search.best_estimator_

        # *** Added Debugging ***
        # Checking if the model is actually fitted before predictions
        try:
            if not hasattr(best_clf, "n_estimators"):
                raise NotFittedError("Model is not properly fitted. Ensure fit() has been called.")
            print("Model is fitted. Proceeding with predictions.")

            # Predict using the fitted model
            y_pred = best_clf.predict(X_test_cv)
            y_proba = best_clf.predict_proba(X_test_cv)[:, 1]

            # Calculate and print performance metrics
            accuracy = accuracy_score(y_test_cv, y_pred)
            precision = precision_score(y_test_cv, y_pred, pos_label='Sensitive')
            recall = recall_score(y_test_cv, y_pred, pos_label='Sensitive')
            f1 = f1_score(y_test_cv, y_pred, pos_label='Sensitive')

            # Check for the ROC AUC score (only if y_test_cv has more than one class)
            roc_auc = 0
            if (np.unique(y_test_cv).shape[0] > 1) and (np.unique(y_proba).shape[0] > 1):
                roc_auc = roc_auc_score(y_test_cv, y_proba)

            print("Accuracy :", accuracy)
            print("***************************")

            # Append results to lists for later aggregation
            ls_accuracy.append(accuracy)
            ls_precision.append(precision)
            ls_recall.append(recall)
            ls_f1.append(f1)
            ls_roc_auc.append(roc_auc)

        except NotFittedError as e:
            print(f"Error: {e}")
            continue  # Skip the current fold if model is not fitted

    # Print aggregated results
    print("--------------------------------------------------")
    print("Mean accuracy: ", np.mean(np.array(ls_accuracy)))
    print("Stdev accuracy: ", statistics.stdev(ls_accuracy))
    print("--------------------------------------------------")
    print("Mean precision: ", np.mean(np.array(ls_precision)))
    print("Stdev precision: ", statistics.stdev(ls_precision))
    print("--------------------------------------------------")
    print("Mean recall: ", np.mean(np.array(ls_recall)))
    print("Stdev recall: ", statistics.stdev(ls_recall))
    print("--------------------------------------------------")
    print("Mean f1: ", np.mean(np.array(ls_f1)))
    print("Stdev f1: ", statistics.stdev(ls_f1))
    print("--------------------------------------------------")
    print("Mean roc_auc: ", np.mean(np.array(ls_roc_auc)))
    print("Stdev roc_auc: ", statistics.stdev(ls_roc_auc))

X_train shape: (23, 4000000)
y_train shape: (23,)
Unique labels in y_train: ['Resistant' 'Sensitive']
#####################################################
----- CV ---  1
Fitting 5 folds for each of 36 candidates, totalling 180 fits


python(71357) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71358) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71359) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71360) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71361) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71362) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71363) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(71364) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Best parameters found:  {'max_depth': 1, 'min_samples_split': 10, 'n_estimators': 4}
Best cross-validation score: 0.77
Model is fitted. Proceeding with predictions.
Accuracy : 0.4
***************************
#####################################################
----- CV ---  2
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found:  {'max_depth': 1, 'min_samples_split': 10, 'n_estimators': 8}
Best cross-validation score: 0.62
Model is fitted. Proceeding with predictions.
Accuracy : 1.0
***************************
#####################################################
----- CV ---  3
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters found:  {'max_depth': 1, 'min_samples_split': 10, 'n_estimators': 2}
Best cross-validation score: 0.72
Model is fitted. Proceeding with predictions.
Accuracy : 0.4
***************************
#####################################################
----- CV ---  4
Fitting 5 folds for each of 36 candidates,

In [42]:
fts_imp=best_clf.feature_importances_

In [43]:
fts_imp.shape

(4000000,)

In [44]:
fts_imp.max()

0.125

In [45]:
[_ for _ in fts_imp if _>0]

[0.02609890109890109,
 0.023989898989898988,
 0.125,
 0.09890109890109891,
 0.02609890109890109,
 0.031666666666666676,
 0.10101010101010101,
 0.10124999999999999,
 0.09890109890109891,
 0.02827380952380953,
 0.09672619047619047,
 0.09333333333333332,
 0.023750000000000007,
 0.125]

In [46]:
imp_index=np.where(fts_imp>0)

In [47]:
imp_index

(array([ 549693,  833224, 1355951, 1417687, 1970322, 2515049, 2750339,
        2807868, 3184620, 3240214, 3327960, 3587847, 3609338, 3639948]),)

In [48]:
encoded_sequences.shape[1]/4

1000000.0

In [49]:
counter=0
i=0

ls_i=[]

while(counter<encoded_sequences.shape[1]):
    
    counter_old=counter
    counter=counter+4
    
    for g in range(4):
        # print("n")
        _
    
    for k in imp_index[0]:

        if (k>=counter_old and k<=counter):
            ls_i.append(i)
            
    i=i+1
            
    

In [50]:
ls_i

[137423,
 208305,
 208306,
 338987,
 354421,
 492580,
 628762,
 687584,
 701966,
 701967,
 796154,
 796155,
 810053,
 831989,
 831990,
 896961,
 902334,
 909986,
 909987]

In [51]:
dict_f={}
for f in ls_i:
    dict_f[f]=[]
    
for seq_dict in df_dict:
    if seq_dict['sequence_id'] in labels_dict:
        for f in ls_i:
            dict_f[f].append(seq_dict['sequence'].upper()[f])

In [52]:
display(dict_f)

{137423: ['A',
  'T',
  'C',
  'A',
  'T',
  'A',
  'G',
  'T',
  'A',
  'A',
  'A',
  'T',
  'C',
  'A',
  'T',
  'T',
  'T',
  'A',
  'A',
  'A',
  'G',
  'A',
  'G',
  'T',
  'T',
  'C',
  'G',
  'C',
  'T'],
 208305: ['A',
  'T',
  'A',
  'C',
  'T',
  'A',
  'A',
  'C',
  'T',
  'T',
  'T',
  'A',
  'C',
  'T',
  'G',
  'A',
  'C',
  'T',
  'A',
  'A',
  'A',
  'A',
  'A',
  'T',
  'A',
  'T',
  'A',
  'A',
  'A'],
 208306: ['G',
  'T',
  'T',
  'G',
  'A',
  'T',
  'T',
  'A',
  'A',
  'G',
  'G',
  'C',
  'G',
  'T',
  'A',
  'T',
  'T',
  'T',
  'C',
  'T',
  'A',
  'A',
  'T',
  'G',
  'G',
  'A',
  'T',
  'T',
  'A'],
 338987: ['A',
  'A',
  'T',
  'C',
  'G',
  'G',
  'T',
  'A',
  'T',
  'A',
  'A',
  'A',
  'T',
  'A',
  'A',
  'T',
  'C',
  'A',
  'T',
  'T',
  'T',
  'T',
  'T',
  'A',
  'A',
  'A',
  'A',
  'T',
  'T'],
 354421: ['C',
  'T',
  'A',
  'T',
  'T',
  'T',
  'C',
  'T',
  'C',
  'T',
  'T',
  'T',
  'A',
  'A',
  'T',
  'T',
  'C',
  'T',
  'T',
  'A',
  'T

In [53]:
import pickle

# # # Assume X_train and y_train are already defined
# # # X_train: Features for training
# # # y_train: Labels for training

# # # Step 1: Create and fit the RandomForest model
model = RandomForestClassifier()

# # # Step 2: Fit the model with training data
state_changed = model.fit(X_train, y_train)

# # # Step 3: Save the fitted model
try:
    # with open('/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/pickle_files/quinine_phenotype.pkl', 'wb') as f:
    with open('../pickle_files/quinine_phenotype.pkl', 'wb') as f:
        pickle.dump(state_changed, f)
    print("Model saved with pickle.")
except Exception as e:
    print(f"Error saving model with pickle: {e}")



Error saving model with pickle: [Errno 20] Not a directory: '../pickle_files/quinine_phenotype.pkl'


In [54]:

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(dict_f)

# Save the DataFrame as a CSV file
# df.to_csv('datadict.csv', index=False)
# df.to_csv('/opt/anaconda3/envs/ML-Project-on-Resistance-Prediction/resistance-prediction/datadict_files/quinine_phenotype_datadict.csv', index=False)
df.to_csv('../datadict_files/quinine_phenotype_datadict.csv', index=False)


print("Dictionary saved as 'quinine_phenotype_datadict.csv'")



OSError: Cannot save file into a non-existent directory: '../datadict_files'